In [1]:
from moku.instruments import PIDController
from moku.instruments import Oscilloscope
from numpy import sqrt, pi, array, arange, polyfit

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import datetime

%matplotlib inline

## YUN calibration

In [ ]:
data = np.array([255, 4.91],[191,],[127,],[64,],[0,0])

## Pressure conversion

In [2]:
def con_pv(voltage):
    return 10**((voltage-7.75)/0.75)

In [3]:
con_pv(2.485)

9.549925860214349e-08

# RedPitaya

In [2]:
import pyrpl, time

DEBUG:pyrpl.attributes:SelectProperty unit of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty unit of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty default_sweep_output of module None has no options!
DEBUG:pyrpl.attributes:SelectProperty defa

In [3]:
p = pyrpl.Pyrpl(hostname="10.0.2.111", config="111.yml")

INFO:pyrpl:Please select or create a configuration file in the file selector window!
DEBUG:pyrpl.memory:Loading config file C:/Users/Ingrid/pyrpl_user_dir/config\global_config.yml
INFO:pyrpl:All your PyRPL settings will be saved to the config file
    C:/Users/Ingrid/pyrpl_user_dir/config\global_config.yml
If you would like to restart PyRPL with these settings, type "pyrpl.exe global_config" in a windows terminal or 
    from pyrpl import Pyrpl
    p = Pyrpl('global_config')
in a python terminal.


ExpectedPyrplError: 
Could not connect to the Red Pitaya device with the following parameters: 

	hostname: 129.206.183.68
	ssh port: 22
	username: root
	password: ****

Please confirm that the device is reachable by typing its hostname/ip address into a web browser and checking that a page is displayed. 

Error message: timed out[35m


        An expected error occured in PyRPL. Please follow the instructions
        in this error message and retry! [0m

In [ ]:
# Access the RedPitaya object in charge of communicating with the board
r = p.rp

#check the value of input1
print r.scope.voltage1

In [ ]:
#see how the adc reading fluctuates over time
import time
from matplotlib import pyplot as plt
times, data = [],[]
t0 = time.time()
n = 3000
for i in range(n):
    times.append(time.time()-t0)
    data.append(r.scope.voltage_in1)
print("Rough time to read one FPGA register: ", (time.time()-t0)/n*1e6, "?s")
%matplotlib inline
f, axarr = plt.subplots(1,2, sharey=True)
axarr[0].plot(times, data, "+")
axarr[0].set_title("ADC voltage vs time")
axarr[1].hist(data, bins=10,normed=True, orientation="horizontal")
axarr[1].set_title("ADC voltage histogram")

### Oscilloscope

In [ ]:
s = r.scope # shortcut
print("Available decimation factors:", s.decimations)
print("Trigger sources:", s.trigger_sources)
print("Available inputs: ", s.inputs)

In [ ]:
asg = r.asg1
s = r.scope

# turn off asg so the scope has a chance to measure its "off-state" as well
asg.output_direct = "off"

# setup scope
s.input1 = 'asg1'

# pass asg signal through pid0 with a simple integrator - just for fun (detailed explanations for pid will follow)
r.pid0.input = 'asg1'
r.pid0.ival = 0 # reset the integrator to zero
r.pid0.i = 1000 # unity gain frequency of 1000 hz
r.pid0.p = 1.0 # proportional gain of 1.0
r.pid0.inputfilter = [0,0,0,0] # leave input filter disabled for now

# show pid output on channel2
s.input2 = 'pid0'

# trig at zero volt crossing
s.threshold_ch1 = 0

# positive/negative slope is detected by waiting for input to
# sweep through hysteresis around the trigger threshold in
# the right direction
s.hysteresis_ch1 = 0.01

# trigger on the input signal positive slope
s.trigger_source = 'ch1_positive_edge'

# take data symetrically around the trigger event
s.trigger_delay = 0

# set decimation factor to 64 -> full scope trace is 8ns * 2^14 * decimation = 8.3 ms long
s.decimation = 64

# launch a single (asynchronous) curve acquisition, the asynchronous
# acquisition means that the function returns immediately, eventhough the
# data-acquisition is still going on.
res = s.curve_async()

print("Before turning on asg:")
print("Curve ready:", s.curve_ready()) # trigger should still be armed

# turn on asg and leave enough time for the scope to record the data
asg.setup(frequency=1e3, amplitude=0.3, start_phase=90, waveform='halframp', trigger_source='immediately')
sleep(0.010)

# check that the trigger has been disarmed
print("After turning on asg:")
print("Curve ready:", s.curve_ready())
print("Trigger event age [ms]:",8e-9*((
s.current_timestamp&0xFFFFFFFFFFFFFFFF) - s.trigger_timestamp)*1000)

# The function curve_async returns a *future* (or promise) of the curve. To
# access the actual curve, use result()
ch1, ch2 = res.result()

# plot the data
%matplotlib inline
plt.plot(s.times*1e3, ch1, s.times*1e3, ch2)
plt.xlabel("Time [ms]")
plt.ylabel("Voltage")

In [ ]:
# useful functions for scope diagnostics
print("Curve ready:", s.curve_ready())
print("Trigger source:",s.trigger_source)
print("Trigger threshold [V]:",s.threshold_ch1)
print("Averaging:",s.average)
print("Trigger delay [s]:",s.trigger_delay)
print("Trace duration [s]: ",s.duration)
print("Trigger hysteresis [V]", s.hysteresis_ch1)
print("Current scope time [cycles]:",hex(s.current_timestamp))
print("Trigger time [cycles]:",hex(s.trigger_timestamp))
print("Current voltage on channel 1 [V]:", r.scope.voltage_in1)
print("First point in data buffer 1 [V]:", s.ch1_firstpoint)

# Stability Measurements

In [ ]:
# ----- data ----- #
data = np.flip(np.loadtxt('nakayun2_03_10_2021.csv', delimiter=',', skiprows=1, usecols=[1,2]), axis=0)
time = np.flip(np.loadtxt('nakayun2_03_10_2021.csv', delimiter=',', skiprows=1, usecols=0, dtype='str'))

labels=['trash', 'spec']; col = list(mcolors.TABLEAU_COLORS)

In [ ]:
print(time)
print(data)

In [ ]:
means = []; stds = []
fig, ax = plt.subplots(figsize=(15,5))

# ----- calculate ----- #
for i in range(len(data[0,:])):
    mean = np.round(np.mean(data[:,i]),3); std = np.round(np.std(data[:,i]),3);means.append(mean); stds.append(std)
    
    # ax.text(1,0.8-i*0.3, labels[i] +': \n mean = ' + str(mean) + ' V \n std = ' + str(std), fontsize=14, transform=ax.transAxes, bbox=props)
    ax.plot(time, data[:,i], label=labels[i]+': \n mean = ' + str(mean) + ' V \n std = ' + str(std), c=col[i])

# ----- plot ----- #
ax.legend(loc=(1,0), fontsize=12); ax.set_xlabel('time'); ax.set_ylabel('voltage')
# ax.set_xticks(np.arange(0, len(time), int(len(time)/10)))
# ax.set_xticklabels(time[np.arange(0, len(data), int(len(data)/10))], rotation=45)
ax.set_title('01-03/10/2021 - Stability test (trash, spectroscopy)', fontsize=20)

# MOKU:GO

In [ ]:
# This example demonstrates how you can configure the Oscilloscope instrument,
# and view triggered time-voltage data frames in real-time.
#
# (c) 2021 Liquid Instruments Pty. Ltd.

print('START : ' + datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S"))
def on_close(event):
    print('Closed Figure!')
    i.relinquish_ownership()
    
# Connect to your Moku by its ip address using Oscilloscope('192.168.###.###')
# or by its serial number using Oscilloscope(serial=123)
i = Oscilloscope('129.206.180.142', force_connect=True)

try:
    # Trigger on input Channel 1, rising edge, 0V 
    i.set_trigger(type='Edge', source='Input1', level=0)

    # View +-5usec, i.e. trigger in the centre
    i.set_timebase(-5e-6, 5e-6)

    # Generate an output sine wave on Channel 1, 1Vpp, 1MHz, 0V offset
    i.generate_waveform(1, 'DC', amplitude=1)
    i.generate_waveform(2, 'Sine', amplitude=1, frequency=1e6)

    # Set the data source of Channel 1 to be Input 1
    i.set_source(1, 'Input1')

    # Set the data source of Channel 2 to the generated output sinewave
    i.set_source(2, 'Input2')


    # Get initial data frame to set up plotting parameters. This can be done
    # once if we know that the axes aren't going to change (otherwise we'd do
    # this in the loop)
    data = i.get_data()
    print(data.keys())

    # Set up the plotting parameters
    plt.ion()
    plt.show()
    plt.grid(b=True)
    plt.ylim([-1, 1])
    plt.xlim([data['time'][0], data['time'][-1]])

    line1, = plt.plot([], label='channel 1')
    line2, = plt.plot([], label='channel 2')

    # Configure labels for axes
    ax = plt.gca()
    
    # This loops continuously updates the plot with new data
    while True:
        # Get new data
        data = i.get_data()

        # Update the plot
        line1.set_ydata(data['ch1'])
        line2.set_ydata(data['ch2'])
        line1.set_xdata(data['time'])
        line2.set_xdata(data['time'])
        
        ax.set_title(datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S"))
        ax.legend()

        plt.pause(0.001)
        
        fig = plt.gcf()  # get current figure
        fig.canvas.mpl_connect('close_event', on_close)
        
except Exception as e:
    print(f'Exception occurred: {e}')
except KeyboardInterrupt:
    print('Interrupt!')
finally:
    # Close the connection to the Moku device
    # This ensures network resources and released correctly
    i.relinquish_ownership()
    print('Ownership relinquished!')

In [ ]:
i.relinquish_ownership()